In [1]:
import os
import re
import json
import warnings
import pandas as pd
pd.options.mode.chained_assignment = None
from sec_api import QueryApi, XbrlApi

In [ ]:
api_key = "e3d46df2271d81a4b7a71153aef74fa5e7dae956314122b155767267425b36a6"

# Query 20 most recent 10Q Info

In [ ]:
# queryAPI = QueryApi(api_key=api_key)

In [ ]:
# query = {"query": {
#             "query_string": {
#                 "query": "ticker:TSLA AND formType:\"10-Q\"",
#                     }
#                 },
#          "from": "0",
#          "size": 4*5,
#          "sort": [{"filedAt": {"order": "desc"}}]
#             }

In [ ]:
### NOTE: USES UP MONTHLY API CALLS LIMIT
# response = queryAPI.get_filings(query)
# response_parsed = pd.DataFrame.from_records(response["filings"])

In [ ]:
# response_parsed.to_csv("TSLA_10Q_INFO_2017Q1_2023Q2.csv", index=False)
response_parsed = pd.read_csv("TSLA_10Q_INFO_2017Q1_2023Q2.csv")

# Parse 10Q

In [ ]:
# xbrlAPI = XbrlApi(api_key)

In [ ]:
# %%time
### NOTE: USES UP MONTHLY API CALLS LIMIT
# for date, url in response_parsed[["periodOfReport", "linkToFilingDetails"]].values:
#     xbrl_json = xbrlAPI.xbrl_to_json(url)
    
#     with open(f"tesla_10Q_{date}.json", "w") as f:
#         json.dump(xbrl_json, f)

## Extract Data (Dev)

In [ ]:
with open("tesla_10Q_2023-06-30.json", "r") as f:
    xbrl_json = json.load(f)

In [ ]:
current_date = xbrl_json["CoverPage"]["DocumentPeriodEndDate"]
current_date

In [ ]:
BalanceSheets = xbrl_json["BalanceSheets"]
StatementsOfIncome = xbrl_json["StatementsOfIncome"]
StatementsOfCashFlows = xbrl_json["StatementsOfCashFlows"]
StatementsOfShareholdersEquity = xbrl_json["StatementsOfShareholdersEquity"]

## Balance Sheets

In [4]:
def balancesheets_to_df(BalanceSheets):
    """
    """
    df_list = []

    for name in BalanceSheets.keys():
        try:
            df = pd.DataFrame.from_records(BalanceSheets[name])
            df["period"] = df["period"].apply(lambda x: x["instant"])
            df["name"] = name
            df_list.append(df)
        except Exception as ex:
            print(name, "---", ex)
    
    return pd.concat(df_list, axis=0)

In [ ]:
balancesheet_df = balancesheets_to_df(BalanceSheets)
set(BalanceSheets.keys()).difference(set(balancesheet_df.name))

In [ ]:
(balancesheet_df.query("segment.isna()").value_counts(["name", "period"]) == 1).mean()

In [ ]:
# balancesheet_df.query("segment.isna() and period == @current_date")

## Statements of Income

In [5]:
def incomestatement_to_df(StatementsOfIncome):
    """
    """
    df_list = []

    for name in StatementsOfIncome.keys():
        try:
            df = pd.DataFrame.from_records(StatementsOfIncome[name])
            df["startDate"] = df["period"].apply(lambda x: x["startDate"])
            df["endDate"] = df["period"].apply(lambda x: x["endDate"])
            df["name"] = name
            df_list.append(df)
        except Exception as ex:
            print(name, "---", ex)
    
    return pd.concat(df_list, axis=0)

In [ ]:
income_df = incomestatement_to_df(StatementsOfIncome)
set(StatementsOfIncome.keys()).difference(set(income_df.name))

In [ ]:
(income_df.query("segment.isna()").value_counts(["name", "startDate", "endDate"]) == 1).mean()

In [ ]:
(income_df.query("segment.isna()").value_counts(["endDate", "name"]) == 2).mean()

In [ ]:
# (income_df.query("segment.isna() and endDate == @current_date")
#  .sort_values(["name", "startDate"]).groupby(["name"]).tail(1))

## Extract Data (prod.)

In [2]:
files = [i for i in os.listdir() if re.match(".*.json$", i)]

In [15]:
%%time
bs_lst, inc_lst = [], []

for file in files:
    with open(file, "r") as f:
        xbrl_json = json.load(f)
    
    current_date = xbrl_json["CoverPage"]["DocumentPeriodEndDate"]
    BalanceSheets = xbrl_json["BalanceSheets"]
    StatementsOfIncome = xbrl_json["StatementsOfIncome"]
    
    balancesheet_df = balancesheets_to_df(BalanceSheets)
    balancesheet_df = balancesheet_df.query("segment.isna() and period == @current_date")
    bs_lst.append(balancesheet_df)
    
    income_df = incomestatement_to_df(StatementsOfIncome)
    income_df = (income_df
                 .query("segment.isna() and endDate == @current_date")
                 .sort_values(["name", "startDate"])
                 .groupby(["name"])
                 .tail(1))
    inc_lst.append(income_df)

Goodwill --- string indices must be integers
NetIncomeLossAttributableToNoncontrollingInterest --- string indices must be integers
TemporaryEquityCarryingAmountAttributableToParent --- string indices must be integers
TemporaryEquityCarryingAmountAttributableToParent --- string indices must be integers
DueToRelatedPartiesCurrent --- string indices must be integers
RestructuringAndOtherExpenses --- string indices must be integers
TemporaryEquityCarryingAmountAttributableToParent --- string indices must be integers
RestructuringAndOtherExpenses --- string indices must be integers
TemporaryEquityCarryingAmountAttributableToParent --- string indices must be integers
TemporaryEquityCarryingAmountAttributableToParent --- string indices must be integers
RestructuringAndOtherExpenses --- string indices must be integers
MarketableSecuritiesCurrent --- string indices must be integers
CPU times: total: 3.44 s
Wall time: 3.91 s


In [16]:
balancesheet_df = pd.concat(bs_lst, axis=0)
income_df = pd.concat(inc_lst, axis=0)

In [21]:
income_df.dropna(subset=["value"]).value_counts(["endDate"]).sort_index()

endDate   
2017-03-31    26
2017-06-30    27
2017-09-30    29
2018-03-31    29
2018-06-30    30
2018-09-30    27
2019-03-31    25
2019-06-30    26
2019-09-30    26
2020-03-31    25
2020-06-30    25
2020-09-30    27
2021-03-31    20
2021-06-30    20
2021-09-30    21
2022-03-31    20
2022-06-30    20
2022-09-30    20
2023-03-31    19
2023-06-30    20
dtype: int64

In [19]:
balancesheet_df.dropna(subset=["value"]).value_counts(["period"]).sort_index()

period    
2017-03-31    34
2017-06-30    35
2017-09-30    35
2018-03-31    35
2018-06-30    35
2018-09-30    34
2019-03-31    34
2019-06-30    35
2019-09-30    34
2020-03-31    29
2020-06-30    29
2020-09-30    29
2021-03-31    30
2021-06-30    30
2021-09-30    30
2022-03-31    32
2022-06-30    32
2022-09-30    32
2023-03-31    32
2023-06-30    32
dtype: int64